In [18]:
import requests
import base64
from bs4 import BeautifulSoup
import pandas
import sqlite3 as lite
from shutil import copyfileobj
import smtplib
import mimetypes
from email.mime.multipart import MIMEMultipart
from email import encoders
from email.message import Message
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from datetime import datetime

#get time of day 
currenttime = datetime.now()

#gmail setting
emailfrom = "gghammer@gmail.com"
emailto = "gghammer@gmail.com"
fileToSend = "daily.xlsx"
username = "gghammer"
password = "ggpudin81161"

#payload data for twse
payload={}

#catch payload 
res = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_d.php#')
res.encoding = 'big5'

#find key & Encryption 
soup    = BeautifulSoup(res.text, 'html.parser') 
for inp in soup.select('input'):
    if 'hidden' in inp.get('type'):
        payload[inp.get('name')] = base64.b64encode(inp.get('value').encode('utf-8'))       
#download csw file        
res2 = requests.post('http://www.tse.com.tw/ch/trading/exchange/BWIBBU/BWIBBU_print.php?language=ch&save=csv',data = payload ,stream = True)

#save to twse.csv 
f = open('twse.csv','wb')
copyfileobj (res2.raw,f)
f.close()

#read csw to pandas 
twse = pandas.read_csv('twse.csv',encoding = "big5")

#save to sqlite file 
with lite.connect('twse.sqlite') as conn:
    twse.to_sql(name='twse', con = conn, if_exists='replace' )
    
#save to xlsx file
writer = pandas.ExcelWriter(fileToSend, engine='xlsxwriter')
twse.to_excel(writer, sheet_name='Sheet1')
writer.save()

#gmail send 
msg = MIMEMultipart()
msg["From"] = emailfrom
msg["To"] = emailto
msg["Subject"] = "哈墨投顧日報 "+currenttime.strftime('%Y-%m-%d')
msg.preamble = "help I cannot send an attachment to save my life"

ctype, encoding = mimetypes.guess_type(fileToSend)
if ctype is None or encoding is not None:
    ctype = "application/octet-stream"

maintype, subtype = ctype.split("/", 1)

if maintype == "text":
    fp = open(fileToSend)
    # Note: we should handle calculating the charset
    attachment = MIMEText(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "image":
    fp = open(fileToSend, "rb")
    attachment = MIMEImage(fp.read(), _subtype=subtype)
    fp.close()
elif maintype == "audio":
    fp = open(fileToSend, "rb")
    attachment = MIMEAudio(fp.read(), _subtype=subtype)
    fp.close()
else:
    fp = open(fileToSend, "rb")
    attachment = MIMEBase(maintype, subtype)
    attachment.set_payload(fp.read())
    fp.close()
    encoders.encode_base64(attachment)
attachment.add_header("Content-Disposition", "attachment", filename=fileToSend)
msg.attach(attachment)

server = smtplib.SMTP("smtp.gmail.com:587")
server.starttls()
server.login(username,password)
server.sendmail(emailfrom, emailto, msg.as_string())
server.quit()

//anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1201: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


(221, b'2.0.0 closing connection u14sm41905436pfg.18 - gsmtp')